아래는 ALO 기본 설정 및 라이브러리 설치 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 하고, 지속적으로 문제가 있을 시 문의바랍니다

In [1]:
import os
import jupyter_supporter as js 
import copy
os.chdir(os.path.abspath(os.path.join('..')))

JS=js.Jsupport()
JS.download_data()

======================================== Start dependency installation - step : << master >> ========================================
>> Start checking existence & installing package - alolib@git+http://mod.lge.com/hub/dxadvtech/aicontents-framework/alolib-source.git@develop | Progress: ( 1 / 1 total packages )
- << alolib@git+http://mod.lge.com/hub/dxadvtech/aicontents-framework/alolib-source.git@develop >> already exists
======================================== Finish dependency installation ======================================== 

======================================== Start dependency installation - step : << master >> ========================================
>> Start checking existence & installing package - alolib@git+http://mod.lge.com/hub/dxadvtech/aicontents-framework/alolib-source.git@develop | Progress: ( 1 / 1 total packages )
- << alolib@git+http://mod.lge.com/hub/dxadvtech/aicontents-framework/alolib-source.git@develop >> already exists
===============================

  Obtaining dependency information for numpy==1.25.2 from https://files.pythonhosted.org/packages/71/3c/3b1981c6a1986adc9ee7db760c0c34ea5b14ac3da9ecfcf1ea2a4ec6c398/numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)


  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
======================================== Finish dependency installation ======================================== 

>> Start removing pre-existing input data before fetching external data: <DirEntry 'inf'>
>> Start removing pre-existing input data before fetching external data: <DirEntry 'train_missing'>
>> s3 private key file << load_s3_key_path >> loaded successfully.
>> Start fetching external data from << /nas001/users/yoonji.suh/tcr_test_20231011/train_missing/ >> into << input >> folder.
>> Start fetching external data from << /nas001/users/yoonji.suh/tcr_test_20231011/inf/ >> into << input >> folder.


## Train workflow 
### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [4]:
train_pipelines = 'train_pipeline'
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 0 
input_args = JS.user_parameters[train_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 

# input_args['x_columns'] = ['']
input_args

{'input_path': 'train_missing',
 'x_columns': ['Count',
  'Converted Torque',
  'angle_min',
  'angle_mean',
  'angle_median',
  'angle_std',
  'angle_max',
  'torque_min',
  'torque_mean',
  'torque_median',
  'torque_std',
  'torque_max'],
 'use_all_x': False,
 'y_column': 'label',
 'groupkey_columns': None,
 'drop_columns': None,
 'time_column': None}

##### Input asset 실행 

In [7]:
data = pipe_val = 0 # 초기 input asset process 세팅
data_input, config_input = JS.run(step, input_args, train_pipelines, data, pipe_val) 
# data_input: input asset의 결과 dataframe입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.
data_input['dataframe'].head(10)

input asset has already been created at 2023-10-15 05:09:02.349005



============================= ASSET INFO =============================
PIPELINES    : train_pipeline
ASSETS       : input



==================== current time : 2023-10-15 07:09:22.862350+00:00 (UTC) ====================
************************************************************
************************************************************
>> Load path : ['/home/jovyan/project2023/dev/231015_1/tcr/alo/input/train_missing/']
>> The file for batch data has been loaded. (File name: /home/jovyan/project2023/dev/231015_1/tcr/alo/input/train_missing/BF_stat.csv)
==================== Success loading dataframe ====================
>> Drop columns from the input dataframe when set << auto >> mode or specified in the << drop_columns >> in config yaml. (dropped colums:[])


,Unnamed: 0,No,Time,Ch1,Ch2,Count,Fastening Time,Target Torque,Converted Torque,Speed,...,angle_median,angle_std,torque_max,torque_min,torque_mean,torque_median,torque_std,ng_type,encoded_label,test
0,0,1.0,16:08:35,1,4,NaN,3207,6.0,6.02,250,...,3469.0,1793.955856,5.95,0.00,1.443754,1.24,1.139063,long,0,0
1,1,1.0,16:08:11,1,4,661.0,3298,6.0,6.02,250,...,3572.0,1828.091133,5.93,-0.10,1.540484,1.44,1.169756,long,0,1
2,2,1.0,16:09:21,1,4,NaN,3469,6.0,6.02,250,...,3731.0,1889.288418,5.93,0.00,1.491612,1.22,1.284394,long,0,2
3,3,1.0,16:09:43,1,4,644.0,3214,6.0,6.08,250,...,3475.0,1800.137459,6.00,0.00,1.521817,1.34,1.110613,long,0,3
4,4,NaN,10:03:30,1,4,518.0,2586,6.0,6.04,250,...,2737.0,1570.715252,5.96,0.00,0.391062,0.08,1.073781,rework,0,4
5,5,1.0,09:38:06,1,4,513.0,2556,6.0,6.00,250,...,2702.0,1563.218318,6.00,-0.09,0.250799,-0.04,1.037297,rework,0,5
6,6,1.0,10:07:58,1,4,499.0,2493,6.0,6.02,250,...,2626.0,1535.521352,5.81,0.00,0.316032,0.06,0.996103,rework,0,6
7,7,1.0,10:10:46,1,4,487.0,2426,6.0,6.00,250,...,2545.0,1509.739724,5.99,0.00,0.315688,0.04,1.065089,rework,0,7
8,8,1.0,09:45:01,1,4,505.0,2517,6.0,6.02,250,...,2651.0,1546.469187,5.92,0.00,0.386891,0.08,1.026302,rework,0,8
9,9,1.0,11:09:31,1,4,503.0,2508,6.0,6.02,250,...,2645.0,1540.671743,6.00,0.00,0.361014,0.06,1.102715,rework,0,9


,Unnamed: 0,No,Time,Ch1,Ch2,Count,Fastening Time,Target Torque,Converted Torque,Speed,...,angle_median,angle_std,torque_max,torque_min,torque_mean,torque_median,torque_std,ng_type,encoded_label,test
0,0,1.0,16:08:35,1,4,NaN,3207,6.0,6.02,250,...,3469.0,1793.955856,5.95,0.00,1.443754,1.24,1.139063,long,0,0
1,1,1.0,16:08:11,1,4,661.0,3298,6.0,6.02,250,...,3572.0,1828.091133,5.93,-0.10,1.540484,1.44,1.169756,long,0,1
2,2,1.0,16:09:21,1,4,NaN,3469,6.0,6.02,250,...,3731.0,1889.288418,5.93,0.00,1.491612,1.22,1.284394,long,0,2
3,3,1.0,16:09:43,1,4,644.0,3214,6.0,6.08,250,...,3475.0,1800.137459,6.00,0.00,1.521817,1.34,1.110613,long,0,3
4,4,NaN,10:03:30,1,4,518.0,2586,6.0,6.04,250,...,2737.0,1570.715252,5.96,0.00,0.391062,0.08,1.073781,rework,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,1.0,11:50:49,1,4,480.0,2396,6.0,6.06,250,...,2442.0,1498.036939,6.02,-0.01,0.988771,0.86,0.944294,none,1,495
496,497,1.0,21:19:06,1,4,495.0,2472,6.0,6.06,250,...,2529.0,1527.882210,5.96,-0.01,1.104586,0.94,0.980097,none,1,496
497,498,1.0,18:59:56,1,4,517.0,2578,6.0,6.04,250,...,2657.0,1574.932713,6.03,0.00,1.114023,0.85,0.967550,none,1,497
498,499,1.0,16:31:59,1,4,519.0,2587,6.0,6.05,250,...,2669.0,1579.921028,5.98,-0.01,1.107495,0.99,0.931000,none,1,498


</br>

### 2. Preprocess asset 
##### Preprocess asset의 args수정 및 확인
- preprocess_args의 항목을 ***preprocess_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 1 
preprocess_args = JS.user_parameters[train_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# preprocess_args['handling_missing'] = 'interpolation'
preprocess_args

##### Preprocess asset 실행 

In [ ]:
data_preprocess, config_preprocess = JS.run(step, preprocess_args, train_pipelines, data_input, config_input) 
# data_preprocess: preprocess asset의 결과 dataframe입니다. 다음 asset 실행 시 필요합니다. 
# config_preprocess: preprocess asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

</br>

### 3. Sampling asset 
##### Sampling asset의 args수정 및 확인
- Sampling_args의 항목을 ***sampling_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 2 
sampling_args = JS.user_parameters[train_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# sampling_args['sampling_type'] = 'none'
sampling_args

##### Sampling asset 실행 

In [ ]:
data_sampling, config_sampling = JS.run(step, sampling_args, train_pipelines, data_preprocess, config_preprocess) 
# data_sampling: sampling asset의 결과 dataframe입니다. 다음 asset 실행 시 필요합니다. 
# config_sampling: sampling asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

</br>

### 4. TCR asset 
##### TCR asset의 args수정 및 확인
- TCR_args의 항목을 ***TCR_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 3 
tcr_args = JS.user_parameters[train_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 tcr_args를 수정합니다. 
# tcr_args['model_list'] = ['lgb','rf']
tcr_args

##### TCR asset 실행 

In [ ]:
data_tcr, config_tcr = JS.run(step, tcr_args, train_pipelines, data_sampling, config_sampling) 
# data_tcr: TCR asset의 결과 dataframe입니다. 
# config_tcr: TCR asset의 결과 config입니다. 

<br>

## Inference workflow 
### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
inf_pipelines = 'inference_pipeline'
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - train(2))
step = 0 
input_args = JS.user_parameters[inf_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 

# input_args['x_columns'] = ['']
input_args

##### Input asset 실행 

In [ ]:
data = pipe_val = 0 # 초기 input asset process 세팅
data_input, config_input = JS.run(step, input_args, inf_pipelines, data, pipe_val) 
# data_input: input asset의 결과 dataframe입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

</br>

### 2. Preprocess asset 
##### Preprocess asset의 args수정 및 확인
- preprocess_args의 항목을 ***preprocess_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - train(2))
step = 1 
preprocess_args = JS.user_parameters[inf_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# preprocess_args['handling_missing'] = 'interpolation'
preprocess_args

##### Preprocess asset 실행 

In [ ]:
data_preprocess, config_preprocess = JS.run(step, preprocess_args, inf_pipelines, data_input, config_input) 
# data_preprocess: preprocess asset의 결과 dataframe입니다. 다음 asset 실행 시 필요합니다. 
# config_preprocess: preprocess asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

<br>

### 3. TCR asset 
##### TCR asset의 args수정 및 확인
- TCR_args의 항목을 ***TCR_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - train(2))
step = 2
tcr_args = JS.user_parameters[inf_pipelines][step]['args'][0].copy()

# 아래 주석을 풀어 tcr_args를 수정합니다. 
# tcr_args['model_list'] = ['lgb','rf']
tcr_args

##### TCR asset 실행 

In [ ]:
data_tcr, config_tcr = JS.run(step, tcr_args, inf_pipelines, data_preprocess, config_preprocess) 
# data_tcr: TCR asset의 결과 dataframe입니다. 
# config_tcr: TCR asset의 결과 config입니다. 